In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pathlib import Path
import os
#new_directory = "/content/drive/MyDrive/DS681_Assignment4/Task1"
new_directory = "/content/drive/MyDrive/DS681_Assignment4"
os.chdir(new_directory)
HOME = Path.cwd()
print("HOME:", HOME)

HOME: /content/drive/MyDrive/DS681_Assignment4


In [ ]:
!pip install git+github.com
!sudo apt update && sudo apt install ffmpeg

ERROR: Invalid requirement: 'git+github.com': Expected end or semicolon (after name and no valid version specifier)
    git+github.com
       ^
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRe

In [ ]:
#this converts the video file to audio file to transcribe
#!sudo ffmpeg -i "Warriors-Lakers-Instant-Classic-2021.mp4" "Warriors-Lakers-Instant-Classic-2021-audio.mp3"

In [ ]:
#installing the whsiper model
!pip install -q git+https://github.com/openai/whisper.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os, re
import torch
from pathlib import Path

import whisper
from whisper.utils import get_writer

# Use CUDA, if available
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load the desired model
model = whisper.load_model("medium.en").to(DEVICE)

100%|██████████████████████████████████████| 1.42G/1.42G [00:05<00:00, 285MiB/s]


In [ ]:
#function to transcribe the audio file
def transcribe_file(model, file, plain, srt,download):
    """
    Transcribe a single audio file using whisper
    """
    file_path = Path(file)
    print(f"Transcribing file: {file_path}\n")

    output_directory = file_path.parent

    # Run Whisper
    result = model.transcribe(file, verbose = False, language = "en")

    if plain:
        txt_path = file_path.with_suffix(".txt")
        print(f"\nCreating text file")

        with open(txt_path, "w", encoding="utf-8") as txt:
            txt.write(result["text"])
    if srt:
        print(f"\nCreating SRT file")
        srt_writer = get_writer("srt", output_directory)
        srt_writer(result, str(file_path.stem))


    if download:
        from google.colab import files

        colab_files = Path("/content/drive/MyDrive/DS681_Assignment4/Task1")
        stem = file_path.stem

        for colab_file in colab_files.glob(f"{stem}*"):
            if colab_file.suffix in [".txt", ".srt"]:
                print(f"Downloading {colab_file}")
                files.download(str(colab_file))

    return result

In [ ]:
input_format = "gdrive"
file =  "/content/drive/MyDrive/DS681_Assignment4/Task1/Warriors-Lakers-Instant-Classic-2021-audio.mp3"
plain = True; #this will save the file as plain text
srt = True; #this will save the file as SRT for time stamps
download = True;

result = transcribe_file(model, file, plain, srt, download)

Transcribing file: /content/drive/MyDrive/DS681_Assignment4/Task1/Warriors-Lakers-Instant-Classic-2021-audio.mp3



100%|██████████| 560597/560597 [11:11<00:00, 834.63frames/s]


Creating text file

Creating SRT file


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#here I am going to install the Llama model for the queries
!pip install transformers accelerate sentencepiece

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `DS681-DeepLearningForComputerVision` has been saved to /root/.cache/huggingface/stored_tokens
Your token has b

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


#going to try Qwen for now as waiting to get approved to use LLama model
model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
transcript  = open("/content/drive/MyDrive/DS681_Assignment4/Task1/Warriors-Lakers-Instant-Classic-2021-audio.srt").read()

In [ ]:
def ask_video_question(transcript, question):
    prompt = f"""
You are an AI system that answers questions about a video transcript.
Provide concise, Google-style analysis.
If timestamps appear, cite them like (12:35).

Transcript:
{transcript}

User question:
{question}
"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    # Store the length of the input prompt tokens
    input_length = inputs.input_ids.shape[1]

    output = model.generate(
        **inputs,
        max_new_tokens=500,
        temperature=0.2
    )
    #this was added so that the LLM does not print out the entire transcript everytime it answers.
    generated_text = tokenizer.decode(output[0][input_length:], skip_special_tokens=True)
    return generated_text

In [ ]:
answer = ask_video_question(transcript, "Summarize the key moments.")
print(answer)

This is a friendly reminder - the current text generation call has exceeded the model's predefined maximum length (32768). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


Key moments from the game include:

1. The Warriors dominated the first half, leading by as much as 13 points. Curry was particularly effective, scoring 18 points in the second quarter alone.

2. In the second half, the Lakers rallied, led by LeBron James and Anthony Davis. Davis had a strong second half with 25 points, including 13 in the fourth quarter.

3. James hit a game-winning three-pointer with 1.1 seconds left to give the Lakers a 104-101 victory.

4. The Lakers' defense was key, limiting the Warriors to just 37% shooting in the second half.

5. Draymond Green had a standout performance, recording 12 rebounds and 10 assists while also drawing multiple fouls.

6. The Lakers' bench contributed significantly, with Jordan Poole scoring 15 points off the bench.

7. Despite Curry's efforts, he was limited to 41 minutes due to the team's need to manage his minutes.

8. The game featured intense physical play, with several controversial calls that affected the outcome.  // The final m

In [ ]:
answer = ask_video_question(transcript, "Analyze the player that scored the most in this game. Can your response be in bullet point form")
print(answer)


You are an AI system that answers questions about a video transcript.
Provide concise, Google-style analysis.
If timestamps appear, cite them like (12:35).

Transcript:
1
00:00:00,000 --> 00:00:01,600
winning streak

2
00:00:01,600 --> 00:00:04,000
And the warriors ended with a six game winning streak

3
00:00:05,520 --> 00:00:10,640
Watch out james again with that right ankle. He played last couple of games did tweak it on sunday

4
00:00:11,600 --> 00:00:14,000
But he played very well in those final two games

5
00:00:15,200 --> 00:00:20,960
And here's one of the fun matchups in this series anthony davis has to take a tough shot. Well defended by draymond green

6
00:00:20,960 --> 00:00:22,960
And

7
00:00:25,600 --> 00:00:31,040
Obviously both teams would love to get the win tonight and give it a little extra time to heal and prepare for their opponent

8
00:00:31,440 --> 00:00:33,840
As andrew wingans gets inside and gets the bucket

9
00:00:35,040 --> 00:00:37,200
To me, he's the

In [ ]:
answer = ask_video_question(transcript, "Can you compare Lebron James vs Stephen Curry performance? Can your response be in bullet point form")
print(answer)

Sure, here's a comparison of LeBron James and Stephen Curry's performances:

- **Scoring:**
  - **Stephen Curry:** Scored 37 points, hitting multiple three-pointers, particularly in the second half.
  - **LeBron James:** Scored 22 points, including a game-winning three-pointer with 1 minute left in the game.

- **Rebounding:**
  - **Stephen Curry:** 5 rebounds.
  - **LeBron James:** 11 rebounds, including several key defensive rebounds.

- **Assists:**
  - **Stephen Curry:** 6 assists.
  - **LeBron James:** 10 assists, demonstrating his playmaking ability.

- **Efficiency:**
  - **Stephen Curry:** High efficiency with 10 of 18 field goals and 6 of 9 three-pointers.
  - **LeBron James:** More balanced with a mix of scoring and playmaking, but slightly lower shooting percentage.

- **Defense:**
  - **Stephen Curry:** Played solid defense, particularly in helping to contain the Lakers' offense.
  - **LeBron James:** Dominated the defensive end, making critical plays and altering shots.

-

In [ ]:
#going to use the llama model llama3.1
model_name = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)



tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
answer = ask_video_question(transcript, "Summarize the key moments.")
print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer:
The key moments of the game include:

* The Warriors' strong start, with Steph Curry making three three-pointers in the second quarter to give them a 13-point lead.
* The Lakers' comeback in the third quarter, with Anthony Davis and the bench contributing to a 7-point lead.
* The Warriors' response in the fourth quarter, with Curry making several clutch shots to tie the game.
* The Lakers' final possession, with LeBron James hitting a game-winning three-pointer with 0.8 seconds remaining.
* The controversy surrounding a flagrant foul call on Draymond Green, which led to James getting two free throws and possession.
* The Warriors' final possession, with Curry missing a three-pointer at the buzzer.

These moments highlight the back-and-forth nature of the game and the clutch performances of both Curry and James. The game ultimately came down to a thrilling finish, with the Lakers emerging victorious.


In [ ]:
answer = ask_video_question(transcript, "Analyze the player that scored the most in this game")
print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The player that scored the most in this game was Steph Curry with 37 points.

Analysis:
The player that scored the most in this game was Steph Curry with 37 points. Curry had a remarkable performance, shooting 10 of 18 from the field and 6 of 9 from three-point range. He was a key factor in the Warriors' offense, scoring 35 points in the second half alone. Curry's ability to score from anywhere on the court and his clutch shooting made him a nightmare for the Lakers to defend. His performance was a testament to his incredible skill and experience, and he was the main reason why the Warriors were able to stay in the game despite the Lakers' strong defense. Curry's 37 points were a game-high, and he was the clear leader of the Warriors' offense throughout the game. His performance will be remembered as one of the most impressive individual efforts of the season. (12:35) 

Note: The timestamp (12:35) refers to the time when the analysis was written, not the time when the event occurred. 


In [ ]:
answer = ask_video_question(transcript, "Can you compare Lebron James vs Stephen Curry performance? Can your response be in bullet point form")
print(answer)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Answer:
Here's a comparison of Lebron James and Stephen Curry's performance in the game:

* **Points:** Lebron James scored 22 points, while Stephen Curry scored 37 points.
* **Three-pointers:** Curry made 5 three-pointers, while James made 1 three-pointer.
* **Rebounds:** James had 11 rebounds, while Curry had 12 rebounds.
* **Assists:** James had 10 assists, while Curry had 9 assists.
* **Turnovers:** Curry had 7 turnovers, while James had 4 turnovers.
* **Field goal percentage:** Curry shot 10-18 from the field, while James shot 8-17 from the field.
* **Three-point percentage:** Curry shot 5-11 from three-point range, while James shot 1-5 from three-point range.
* **Free throw percentage:** Curry shot 12-12 from the free throw line, while James shot 2-2 from the free throw line.
* **Game-winning shot:** James made the game-winning three-pointer with 2.1 seconds remaining in the game.

Overall, both players had strong performances, but Curry's scoring and rebounding numbers were slig